In [ ]:
import torch 
print(torch.__version__)

1.11.0+cu113


In [ ]:
!nvidia-smi

Wed May 18 22:09:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 7.9 MB 13.2 MB/s 
Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 3.5 MB 11.7 MB/s 
Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 2.5 MB 7.8 MB/s 
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-m09jxpjn
  Running command git clone -q https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-m09jxpjn
  Created wheel for torch-geometric: filename=torch_geometric-2.0.5-py3-none-any.whl size=645017 sha256=4135baa3207cc59b6803750f9ecbb9dcbb548b46c3e1cc6fffc2f1ce7b2d68bb
  Stored in directory: /tmp/pip-ephem-wheel-cache-1d946knf/wheels/85/c9/07/7936efecad79b906348a7e9fb644d914160544efa9aa7f4b2b
Successfully built torch-geometric


In [ ]:
!pip install ogb

     |████████████████████████████████| 78 kB 5.0 MB/s 
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=d4d347c1199da2307b1f135e44809958e937638f6727584853c90292c74438cd
  Stored in directory: /root/.cache/pip/wheels/d6/64/cd/32819b511a488e4993f2fab909a95330289c3f4e0f6ef4676d
Successfully built littleutils


In [ ]:
import os
import random
import argparse
import numpy as np
import networkx as nx

import torch
import torch.nn.functional as F
import torch_sparse

from torch import Tensor
from torch.nn import Linear
from torch.utils.data import DataLoader
from torch_geometric.utils import negative_sampling, to_networkx
from typing import Union, Tuple
from torch_geometric.typing import OptPairTensor, Adj, OptTensor, Size
from torch_sparse import SparseTensor, matmul
import torch_geometric.nn.conv as ccgeo
from torch_geometric.nn.conv import MessagePassing

In [ ]:
%%writefile main.py

from torch import Tensor
from torch.nn import Linear
from torch.utils.data import DataLoader
from torch_geometric.utils import negative_sampling, to_networkx
from typing import Union, Tuple
from torch_geometric.typing import OptPairTensor, Adj, OptTensor, Size
from torch_sparse import SparseTensor, matmul
import torch_geometric.nn.conv as ccgeo
from torch_geometric.nn.conv import MessagePassing
from torch.nn.modules.module import Module
from ogb.linkproppred import PygLinkPropPredDataset, Evaluator
import pynvml as pynv
import os
import random
import argparse
import numpy as np
import networkx as nx
import torch.nn as nn
import scipy.sparse as sp
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import math


import torch
import torch.nn.functional as F
import torch_sparse
import sys
##### SCRIPT STARTS HERE #####
#!usr/bin/bash python
# instead of nx.shortest_path_length defaults to dijkstra's shortest path, could try Bellman-Ford or etc. Follow: https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.shortest_paths.generic.shortest_path_length.html


def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def normalize_adj(adj):
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)


def skipgnn_preprocessing(edges, num_nodes):
    edges = edges.T
    features = np.eye(num_nodes)

    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                    shape=(num_nodes, num_nodes),
                    dtype=np.float32)

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)

    #create skip graph
    adj2 = adj.dot(adj)
    adj2 = adj2.sign()

    adj2 = normalize_adj(adj2)
    adj2 = sparse_mx_to_torch_sparse_tensor(adj2)

    adj = adj + sp.eye(adj.shape[0])

    #normalize original graph
    adj = normalize_adj(adj)

    features = torch.FloatTensor(features)
    adj = torch.FloatTensor(np.array(adj.todense()))

    return adj, adj2, features


class Logger(object):
    def __init__(self, runs, info=None):
        self.info = info
        self.results = [[] for _ in range(runs)]

    def add_result(self, run, result):
        assert len(result) == 2
        assert run >= 0 and run < len(self.results)
        self.results[run].append(result)

    def print_statistics(self, run=None):
        f = open("./gdrive/MyDrive/output.txt", "a")
        if run is not None:
            result = 100 * torch.tensor(self.results[run])
            print(result, file=f)
            argmax = result[:, 0].argmax().item()
            print(f'Run {run + 1:02d}:', file=f)
            print(f'Highest Valid: {result[:, 0].max():.2f}', file=f)
            print(f'Highest Test: {result[:, 1].max():.2f}', file=f)
            print(f'   Final Test: {result[argmax, 1]:.2f}', file=f)
        else:
            result = 100 * torch.tensor(self.results)
            best_results = []
            for r in result:
                valid = r[:, 0].max().item()
                test = r[r[:, 0].argmax(), 1].item()
                best_results.append((valid, test))
            best_result = torch.tensor(best_results)
            print(f'All runs:', file=f)
            r = best_result[:, 0]
            print(f'Highest Valid: {r.mean():.4f} ± {r.std():.4f}', file=f)
            r = best_result[:, 1]
            print(f'   Final Test: {r.mean():.4f} ± {r.std():.4f}', file=f)
        f.close()


class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        # if input.shape[0] == self.weight.shape[0]:
        #     support = torch.mul(input, self.weight)
        # else:
        #     support = torch.mm(input, self.weight)

        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'


class SkipGNN(nn.Module):
    def __init__(self, nfeat, nhid1, nhid2, nhid_decode1, dropout):
        super(SkipGNN, self).__init__()
        
        # original graph
        self.o_gc1 = GraphConvolution(nfeat, nhid1)
        self.o_gc2 = GraphConvolution(nhid1, nhid2)
        
        # original graph for skip update
        self.o_gc1_s = GraphConvolution(nhid1, nhid1)
        
        #skip graph
        self.s_gc1 = GraphConvolution(nfeat, nhid1)
        
        #skip graph for original update
        self.s_gc1_o = GraphConvolution(nfeat, nhid1)
        self.s_gc2_o = GraphConvolution(nhid1, nhid2)
       
        self.dropout = dropout
        
    def forward(self, x, o_adj, s_adj):
        o_x = F.relu(self.o_gc1(x, o_adj) + self.s_gc1_o(x, s_adj))       
        s_x = F.relu(self.s_gc1(x, s_adj) + self.o_gc1_s(o_x, o_adj))
        
        o_x = F.dropout(o_x, self.dropout, training = self.training)
        s_x = F.dropout(s_x, self.dropout, training = self.training)
        
        x = self.o_gc2(o_x, o_adj) + self.s_gc2_o(s_x, s_adj)
        
        return x


class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(LinkPredictor, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()

    def forward(self, x_i, x_j):
        x = x_i * x_j
        for lin in self.lins[:-1]:
            x = lin(x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.sigmoid(x)


def train(model, predictor, x, adj, adj_2, split_edge, optimizer, device, batch_size, edge_index):
    model.train()
    predictor.train()
    pos_train_edge = split_edge['train']['edge'].to(device)

    total_loss = total_examples = 0
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size, shuffle=True):
        optimizer.zero_grad()
        # h will be embeddings
        h = model(x, adj, adj_2)
        # print("Model output shape:",h.shape)
        # edge: we get a sampling of positive edge connections between nodes by using perm(list of nodes for the batch we know are positive)
        edge = pos_train_edge[perm].t()
        # we pass the edge list inside embedding h
        # we devide in edge[0] and edge[1] to devide the connected from->to
        pos_out = predictor(h[edge[0]], h[edge[1]])
        pos_loss = -torch.log(pos_out + 1e-15).mean()
        # neg_sampling(we get negative edges from dataset) REAL ONES! they use the full dataset!!! VERY GOOD :)
        edge = negative_sampling(edge_index, num_nodes=x.size(0),
                                 num_neg_samples=perm.size(0), method='dense')

        neg_out = predictor(h[edge[0]], h[edge[1]])
        neg_loss = -torch.log(1 - neg_out + 1e-15).mean()

        loss = pos_loss + neg_loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(x, 1.0)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(predictor.parameters(), 1.0)

        optimizer.step()

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples

    return total_loss / total_examples

@torch.no_grad()
def test(model, predictor, x, adj, adj_2, split_edge, evaluator, batch_size):
    model.eval()
    predictor.eval()

    h = model(x, adj, adj_2)

    pos_valid_edge = split_edge['valid']['edge'].to(x.device)
    neg_valid_edge = split_edge['valid']['edge_neg'].to(x.device)
    pos_test_edge = split_edge['test']['edge'].to(x.device)
    neg_test_edge = split_edge['test']['edge_neg'].to(x.device)

    pos_valid_preds = []
    for perm in DataLoader(range(pos_valid_edge.size(0)), batch_size):
        edge = pos_valid_edge[perm].t()
        pos_valid_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_valid_pred = torch.cat(pos_valid_preds, dim=0)

    neg_valid_preds = []
    for perm in DataLoader(range(neg_valid_edge.size(0)), batch_size):
        edge = neg_valid_edge[perm].t()
        neg_valid_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    neg_valid_pred = torch.cat(neg_valid_preds, dim=0)

    pos_test_preds = []
    for perm in DataLoader(range(pos_test_edge.size(0)), batch_size):
        edge = pos_test_edge[perm].t()
        pos_test_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_test_pred = torch.cat(pos_test_preds, dim=0)

    neg_test_preds = []
    for perm in DataLoader(range(neg_test_edge.size(0)), batch_size):
        edge = neg_test_edge[perm].t()
        neg_test_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    neg_test_pred = torch.cat(neg_test_preds, dim=0)

    results = {}
    for K in [20, 50, 100]:
        evaluator.K = K
        valid_hits = evaluator.eval({
            'y_pred_pos': pos_valid_pred,
            'y_pred_neg': neg_valid_pred,
        })[f'hits@{K}']
        test_hits = evaluator.eval({
            'y_pred_pos': pos_test_pred,
            'y_pred_neg': neg_test_pred,
        })[f'hits@{K}']

        results[f'Hits@{K}'] = (valid_hits, test_hits)

    return results
    

def print_gpu_utilization(position):
    print(position)
    os.system("nvidia-smi")

def main():
    parser = argparse.ArgumentParser(description='Link_Pred_DDI')
    parser.add_argument('--device', type=str, default="cuda:0")
    parser.add_argument('--num_layers', type=int, default=2)
    parser.add_argument('--num_samples', type=int, default=5)
    parser.add_argument('--node_emb', type=int, default=256)
    parser.add_argument('--hidden_channels', type=int, default=256)
    parser.add_argument('--spd_size', type=int, default=500)
    # parser.add_argument('--batch_size', type=int, default=1) 
    # TODO
    parser.add_argument('--batch_size', type=int, default=64 * 1024)
    parser.add_argument('--lr', type=float, default=0.003)
    parser.add_argument('--epochs', type=int, default=400)
    parser.add_argument('--log_steps', type=int, default=1)
    parser.add_argument('--eval_steps', type=int, default=1)
    parser.add_argument('--runs', type=int, default=1)
    parser.add_argument('--hidden1', type=int, default=256,
                        help='Number of hidden units for encoding layer 1.')
    parser.add_argument('--hidden2', type=int, default=256,
                        help='Number of hidden units for encoding layer 2.')
    parser.add_argument('--hidden_decode1', type=int, default=16,
                        help='Number of hidden units for decoding layer 1.')
    parser.add_argument('--dropout', type=float, default=0.3,
                        help='Dropout rate (1 - keep probability).')
    args = parser.parse_args()
    print(args)
    device = args.device if torch.cuda.is_available() else 'cpu'
    device = torch.device(device)
    

    dataset = PygLinkPropPredDataset(name='ogbl-ddi')
    data = dataset[0]
    print(data.size())
    edge_index = data.edge_index.to(device)
    print_gpu_utilization("Start")

    split_edge = dataset.get_edge_split()

    # ---
    # create adj matrices required for SkipGNN
    edges = data.edge_index
    # edges = split_edge['train']['edge']
    adj, adj2, features = skipgnn_preprocessing(edges, data['num_nodes'])
    adj, adj2, features = adj.to(device), adj2.to(device), features
    
    # ---

    model = SkipGNN(nfeat=features.shape[1],
            nhid1=args.hidden1,
            nhid2=args.hidden2,
            nhid_decode1 = args.hidden_decode1,
            dropout=args.dropout).to(device)

    emb = torch.nn.Embedding(data.num_nodes, args.node_emb).to(device)
    predictor = LinkPredictor(args.hidden_channels, args.hidden_channels, 1,
                              args.num_layers+1, args.dropout).to(device)

    print('Number of parameters:',
          sum(p.numel() for p in list(model.parameters()) +
          list(predictor.parameters()) + list(emb.parameters())))

    evaluator = Evaluator(name='ogbl-ddi')
    loggers = {
        'Hits@20': Logger(args.runs, args),
        'Hits@50': Logger(args.runs, args),
        'Hits@100': Logger(args.runs, args),
    }
    for run in range(args.runs):
        torch.cuda.empty_cache()
        random.seed(run)
        torch.manual_seed(run)
        torch.nn.init.xavier_uniform_(emb.weight)
        # model.reset_parameters()
        predictor.reset_parameters()
        optimizer = torch.optim.Adam(
            list(model.parameters()) + list(emb.parameters()) + list(predictor.parameters()), lr=args.lr)
        
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, predictor, emb.weight, adj, adj2, split_edge,
                         optimizer,device, args.batch_size, edge_index)
            if epoch % args.eval_steps == 0:
                results = test(model, predictor, emb.weight, adj, adj2, split_edge, evaluator, args.batch_size)
                for key, result in results.items():
                    loggers[key].add_result(run, result)

                if epoch % args.log_steps == 0:
                    f = open("./gdrive/MyDrive/output.txt", "a")
                    for key, result in results.items():
                        valid_hits, test_hits = result
                        print(key, file=f)
                        print(f'Run: {run + 1:02d}, '
                              f'Epoch: {epoch:02d}, '
                              f'Loss: {loss:.4f}, '
                              f'Valid: {100 * valid_hits:.2f}%, '
                              f'Test: {100 * test_hits:.2f}%', file=f)
                    print('---', file=f)
                    f.close()
        torch.save(model,"save.pth")
        for key in loggers.keys():
            print(key)
            loggers[key].print_statistics(run)

    for key in loggers.keys():
        print(key)
        loggers[key].print_statistics()

if __name__ == "__main__":
    main()

Writing main.py


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!python3 main.py --node_emb 4267 --hidden_channels 256 --num_layers 2 --epochs 1000 --runs 1

Namespace(batch_size=65536, device='cuda:0', dropout=0.3, epochs=1000, eval_steps=1, hidden1=256, hidden2=256, hidden_channels=256, hidden_decode1=16, log_steps=1, lr=0.003, node_emb=4267, num_layers=2, num_samples=5, runs=1, spd_size=500)
Downloaded 0.04 GB: 100% 46/46 [00:05<00:00,  8.25it/s]
Extracting dataset/ddi.zip
Processing...
Loading necessary files...
This might take a while.
Processing graphs...
100% 1/1 [00:00<00:00, 40.14it/s]
Converting graphs into PyG objects...
100% 1/1 [00:00<00:00, 12087.33it/s]
Saving...
Done!
(4267, 4267)
Start
Wed May 18 22:10:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|             